Installing libraries

In [14]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [15]:
data_df = pd.read_csv(r"C:\Users\USER\Documents\MLOPS-MNA-Equipo-17\data\processed\TCGA_GBM_LGG_Mutations_clean.csv") #change path when testing
X=data_df.drop(["Grade","Tumor_Type","IDH1", "TP53", "ATRX"], axis=1)
y=data_df["Grade"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
mlflow.set_experiment("Tumor_Classification")

<Experiment: artifact_location='file:///c:/Users/USER/Documents/MLOPS-MNA-Equipo-17/notebooks/4.-%20Visualizacion/mlruns/389002837380232974', creation_time=1729315583011, experiment_id='389002837380232974', last_update_time=1729315583011, lifecycle_stage='active', name='Tumor_Classification', tags={}>

In [19]:
def train_and_log_model(model, model_name, X_train, X_test, y_train, y_test, params):
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train)
        # Make predictions
        y_pred = model.predict(X_test)
        # Calculate metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted')
        rec = recall_score(y_test, y_pred, average='weighted')
        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"accuracy": acc, "precision": prec, "recall": rec})
        # Log the model
        mlflow.sklearn.log_model(model, artifact_path="models")

Logistic Regression

In [20]:
params_lr = {"C": 1.0, "solver": "liblinear", "random_state": 42}
model_lr = LogisticRegression(**params_lr)

In [21]:
train_and_log_model(
    model=model_lr,
    model_name="Logistic_Regression",
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    y_train=y_train,
    y_test=y_test,
    params=params_lr
)

2024/10/18 23:50:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Decision Tree Classifier

In [22]:
params_dt = {"max_depth": 5, "criterion": "entropy", "random_state": 42}
model_dt = DecisionTreeClassifier(**params_dt)

In [23]:
train_and_log_model(
    model=model_dt,
    model_name="Decision_Tree",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_dt
)

2024/10/18 23:50:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


c. Random Forest Classifier

In [24]:
params_rf = {"n_estimators": 100, "max_depth": 5, "random_state": 42}
model_rf = RandomForestClassifier(**params_rf)

In [25]:
train_and_log_model(
    model=model_rf,
    model_name="Random_Forest",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_rf
)

2024/10/18 23:50:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Compare Results

In [26]:
# Example of logging an artifact
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

fig, ax = plt.subplots()
plot_confusion_matrix(model, X_test_scaled, y_test, ax=ax)
plt.savefig("confusion_matrix.png")
mlflow.log_artifact("confusion_matrix.png")

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\__init__.py)